<a href="https://colab.research.google.com/github/ulialiau22/hometask-for-DA-school/blob/main/HW_3_Julia_Dmitrieva.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/da-python/python-for-data-analytics/blob/main/Lesson3/hometask/hometask_SyntheticRubber.ipynb)

In [1]:
##### При работе в Colab
!pip install yfinance requests

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 62 kB 891 kB/s 
  Attempting uninstall: requests
    Found existing installation: requests 2.23.0
    Uninstalling requests-2.23.0:
      Successfully uninstalled requests-2.23.0


In [2]:
import requests
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn; seaborn.set()
import numpy as np

# Задание 1.

* Вы - менеджер по продажам в нефтехимической компании.
* Вы ведете несколько ключевых клиентов, которые закупают продукты вашей компании.
* Ваши клиенты предпочитают формульный подход к ценообразованию и вам нужно предложить клиентам формулу цены на синтетический каучук, который может быть альтернативой натуральному каучуку с грейдом SMR 20.
* В зависимости от объемов, которые покупают клиенты, им может быть предложена скидка.


### Что нужно сделать?

1. Проанализировать имеющиеся данные.
2. Определить базовую формулу цены на условиях FCA (цена на заводе). То есть как бы выглядела цена на исторических данных.
3. Отобразить на графике.
4. Сделать расчет возможной цены по формуле для каждого из клиентов на условиях DDP (цена с доставкой). Записать все в один эксель файл, на разных листах. Каждый лист - название клиента.
5. Предложить формулу цены каждому из клиентов на условиях DDP (цена с доставкой).
- Создать директорию «для клиентов» и в ней сложить файлы с расчетами.
- Каждый клиент - отдельный файл (любой из docx, xlsx, pptx, pdf)  с именем клиента…

In [3]:
# Из данных есть:
# Затраты на производство
PRODUCTION_COST = 400 # (EUR)

In [4]:
# Котировки натурального каучука доступны на сайте,  https://www.lgm.gov.my
# также у них есть API, пример запроса ниже
#
alldfs = []
for y in ['2021', '2022']:
  for m in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
    url = f"https://www.lgm.gov.my/webv2api/api/rubberprice/month={m}&year={y}"
    res = requests.get(url)
    rj = res.json()
    df = pd.json_normalize(rj)
    alldfs.append(df)

In [5]:
df = pd.concat(alldfs) #соединяем датафреймы
df.drop(columns = ['masa', 'rm'], #удаляем столбец rm, т.к. в таблице есть данные о центах, что и требуется для решения задачи
        index = df[df['grade'] != 'SMR 20'].index, inplace = True) # удаляем из таблицы все строки отличные от SMR 20  
df['date'] = pd.to_datetime(df['date']) 
df = pd.DataFrame(df.groupby('date')['us'].mean()) #группируем цены в USD по датам
rate = yf.Ticker('EURUSD=x').history('2y') 
rate = rate.loc['2021-01-01':] #делаем срез - берем все значения с 01 января 2021
rate['r'] = (rate['Open'] + rate['Close'] + rate['Low'] + rate['High'])/4 
rate.drop(columns = rate.columns[:-1], inplace = True)
df = pd.merge(df, rate, left_index = True, right_index=True, how='left') #объединяем датафреймы по индексу - дата
#Пусть для создания 1 тонны синтетического каучука небходимо 100 кг SMR 20
#Тогда формула цены синтетического каучука будет равна = Цена_SMR_20(за тонну) * 0,1 + PRODUCTION_COST 
df['USD'] = df['us'] / 100 * 1000 #переводим центы за кг в доллары за тонну 
df['EUR'] = df['USD'] / df['r'] #из долларов в евро
df['SynthR_FCA_USD'] = 0.1 * df['USD'] + PRODUCTION_COST * df['r'] #расчитываем среднюю цену на заводе в долларах
df['SynthR_FCA_EUR'] = 0.1 * df['EUR'] + PRODUCTION_COST #расчитываем среднюю цену на заводе в евро
df.drop(columns = df.columns[:-2], inplace = True)
df = df.resample('1M').mean()
df['SynthR_FCA_USD_MA'] = df['SynthR_FCA_USD'].rolling(window=3).mean() #расчитываем скользящую среднюю цену на заводе в долларах
df['SynthR_FCA_EUR_MA'] = df['SynthR_FCA_EUR'].rolling(window=3).mean() #расчитываем скользящую среднюю цену на заводе в евро
df.dropna(inplace=True)
df_eur = df[['SynthR_FCA_EUR', 'SynthR_FCA_EUR_MA']] 
df_usd = df[['SynthR_FCA_USD', 'SynthR_FCA_USD_MA']]

In [6]:
# Расходы на логистику
EU_LOGISTIC_COST_EUR = 30 # в Европу в евро
CN_LOGISTIC_COST_USD = 130 # в Китай в долларах
# * Справочная информация по клиентам(объемы, локации, комментарии)
# location - локация клиента
# volumes - закупаемые объемы
# formula - предпочитаемая формула расчета цены - скользящая средняя или среднее значение за месяц
customers = {
    'Monty':{
        'location':'EU',
        'volumes':200,
        'formula':'moving_average'
    },

    'Triangle':{
        'location':'CN',
        'volumes': 30,
        'formula': 'monthly'
    },
    'Stone':{
        'location':'EU',
        'volumes': 150,
        'formula': 'moving_average'
    },
    'Poly':{
        'location':'EU',
        'volumes': 70,
        'formula': 'monthly'
    }
}
# Скидки
discounts = {tuple(np.arange(101)): 0.01, # 1% 
             tuple(np.arange(101,301)): 0.05, # 5%
             tuple(np.arange(301, 2000)): 0.1}   #10%

In [7]:
def get_discount(volume): #создаем функцию, определяющую размер скидки, в зависимости от объема продажи
  for k in discounts.keys():
    if volume in k:
      return(discounts[k])

writer = pd.ExcelWriter('Client_prices.xlsx') #записываем данные в отдельный Excel файл
for customer in customers.keys():
  otkuda = customers[customer]['location']
  skolko = customers[customer]['volumes']
  kak = customers[customer]['formula']


  if otkuda == 'EU': 
    hist_data = df_eur*get_discount(skolko)+EU_LOGISTIC_COST_EUR

    if kak == 'monthly':
      del hist_data['SynthR_FCA_EUR_MA']
    elif kak == 'moving_average':
      del hist_data['SynthR_FCA_EUR']
    hist_data.columns = ['DDP Synthetic Rubber, EUR/tonn']

  elif otkuda == 'CN':
    hist_data = df_usd*get_discount(skolko)+CN_LOGISTIC_COST_USD

    if kak == 'monthly':
      del hist_data['SynthR_FCA_USD_MA']
    elif kak == 'moving_average':
      del hist_data['SynthR_FCA_USD']
    hist_data.columns = ['DDP Synthetic Rubber, USD/tonn']
  
  hist_data.to_excel(writer, f"{customer}")
writer.save()

# Задание 2. Продолжение

Клиенты согласились на ваше предложение.
### Что нужно сделать?
Вам нужно написать скрипт, с помощью которого можно автоматом рассчитывать цены для всех клиентов одновременно.



* В результате должен быть скрипт, который делает расчет цены для клиента (чтобы данные подтягивались туда сами)

# Задание 3. Поделись с ближним

Вы решили пойти в отпуск на время вашего отсутствия вас будет заменять коллега. Он будет заниматься рассчетом цен для клиентов. Чтобы ему было проще вам нужно завернуть ваши скрипты в модуль, который будет запускать у себя коллега.

### Что нужно сделать?
* Создайте модуль, который можно запускать в ноутбуке подкладывая в ту же директорию
* Хорошенько задокументируйте ВСЁ
* Помните о PEP8

In [8]:
def get_personal_prices(clients, out_file_name):

  #clients - Список клиентов в формате словаря. Ключ у словаря - имя клиента, 
  #значение по ключу клиента - словарь, ключи которого:
  #location, volumes, formula - откуда (EU,CN), сколько тонн, по какой формуле считать
  
  #out_file_name - желаемое имя выходного файла (в конце добавить .xlsx)
  
  import requests
  import yfinance as yf
  import pandas as pd
  import matplotlib.pyplot as plt
  import seaborn; seaborn.set()
  import numpy as np
  # Из данных есть:
  # Затраты на производство
  PRODUCTION_COST = 400 # (EUR)
  # Котировки натурального каучука доступны на сайте,  https://www.lgm.gov.my
  # также у них есть API, пример запроса ниже
  #
  alldfs = []
  for y in ['2021', '2022']:
    for m in ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']:
      url = f"https://www.lgm.gov.my/webv2api/api/rubberprice/month={m}&year={y}"
      res = requests.get(url)
      rj = res.json()
      df = pd.json_normalize(rj)
      alldfs.append(df)
  df = pd.concat(alldfs) #соединяем датафреймы
  df.drop(columns = ['masa', 'rm'], #удаляем столбец rm, т.к. в таблице есть данные о центах, что и требуется для решения задачи
          index = df[df['grade'] != 'SMR 20'].index, inplace = True) # удаляем из таблицы все строки отличные от SMR 20  
  df['date'] = pd.to_datetime(df['date']) 
  df = pd.DataFrame(df.groupby('date')['us'].mean()) #группируем цены в USD по датам
  rate = yf.Ticker('EURUSD=x').history('2y') 
  rate = rate.loc['2021-01-01':] #делаем срез - берем все значения с 01 января 2021
  rate['r'] = (rate['Open'] + rate['Close'] + rate['Low'] + rate['High'])/4 
  rate.drop(columns = rate.columns[:-1], inplace = True)
  df = pd.merge(df, rate, left_index = True, right_index=True, how='left') #объединяем датафреймы по индексу - дата
  #Пусть для создания 1 тонны синтетического каучука небходимо 100 кг SMR 20
  #Тогда формула цены синтетического каучука будет равна = Цена_SMR_20(за тонну) * 0,1 + PRODUCTION_COST 
  df['USD'] = df['us'] / 100 * 1000 #переводим центы за кг в доллары за тонну 
  df['EUR'] = df['USD'] / df['r'] #из долларов в евро
  df['SynthR_FCA_USD'] = 0.1 * df['USD'] + PRODUCTION_COST * df['r'] #расчитываем среднюю цену на заводе в долларах
  df['SynthR_FCA_EUR'] = 0.1 * df['EUR'] + PRODUCTION_COST #расчитываем среднюю цену на заводе в евро
  df.drop(columns = df.columns[:-2], inplace = True)
  df = df.resample('1M').mean()
  df['SynthR_FCA_USD_MA'] = df['SynthR_FCA_USD'].rolling(window=3).mean() #расчитываем скользящую среднюю цену на заводе в долларах
  df['SynthR_FCA_EUR_MA'] = df['SynthR_FCA_EUR'].rolling(window=3).mean() #расчитываем скользящую среднюю цену на заводе в евро
  df.dropna(inplace=True)
  df_eur = df[['SynthR_FCA_EUR', 'SynthR_FCA_EUR_MA']] 
  df_usd = df[['SynthR_FCA_USD', 'SynthR_FCA_USD_MA']]
  # Расходы на логистику
  EU_LOGISTIC_COST_EUR = 30 # в Европу в евро
  CN_LOGISTIC_COST_USD = 130 # в Китай в долларах
  # * Справочная информация по клиентам(объемы, локации, комментарии)
  # location - локация клиента
  # volumes - закупаемые объемы
  # formula - предпочитаемая формула расчета цены - скользящая средняя или среднее значение за месяц
  # Скидки
  discounts = {tuple(np.arange(101)): 0.01, # 1% 
              tuple(np.arange(101,301)): 0.05, # 5%
              tuple(np.arange(301, 2000)): 0.1}   #10%

  def get_discount(volume): #создаем функцию, определяющую размер скидки, в зависимости от объема продажи
    for k in discounts.keys():
      if volume in k:
        return(discounts[k])

  writer = pd.ExcelWriter(out_file_name) #записываем данные в отдельный Excel файл
  for customer in clients.keys():
    otkuda = clients[customer]['location']
    skolko = clients[customer]['volumes']
    kak = clients[customer]['formula']


    if otkuda == 'EU': 
      hist_data = df_eur*get_discount(skolko)+EU_LOGISTIC_COST_EUR

      if kak == 'monthly':
        del hist_data['SynthR_FCA_EUR_MA']
      elif kak == 'moving_average':
        del hist_data['SynthR_FCA_EUR']
      hist_data.columns = ['DDP Synthetic Rubber, EUR/tonn']

    elif otkuda == 'CN':
      hist_data = df_usd*get_discount(skolko)+CN_LOGISTIC_COST_USD

      if kak == 'monthly':
        del hist_data['SynthR_FCA_USD_MA']
      elif kak == 'moving_average':
        del hist_data['SynthR_FCA_USD']
      hist_data.columns = ['DDP Synthetic Rubber, USD/tonn']
    
    hist_data.to_excel(writer, f"{customer}")
  writer.save()

customers1 = {
      'Popty':{
          'location':'EU',
          'volumes':200,
          'formula':'moving_average'
      },

      'Goblin':{
          'location':'CN',
          'volumes': 30,
          'formula': 'monthly'
      },
      'Voblin':{
          'location':'EU',
          'volumes': 150,
          'formula': 'moving_average'
      },
      'Dublin':{
          'location':'EU',
          'volumes': 70,
          'formula': 'monthly'
      }
  }
out_file_name = 'ДЗ_3_проверка.xlsx'
get_personal_prices(customers1, out_file_name)

# Задание 4. Поделись пакетом

Ваши коллеги узнали, что вы быстро рассчитываете цены для формульных контрактов, и решили использовать такую же практику для своих продуктов и клиентов. Вам нужно создать пакет, чтобы его было проще устанавливать и использовать.

### Что нужно сделать?
* Создайте отдельный пакет, который можно устанавливать с помощью pip
* В пакете должны быть реализованы функции для рассчета цен по формульным контрактам
* Выложите пакет в свой репозиторий

In [9]:
#Работаю в коллабе, научилась делать с помощью .py файликов в другой IDE